In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric, DatasetDict
import csv
import sentencepiece
import google.protobuf
import evaluate
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
import torch
from tqdm import tqdm
import os

In [2]:
classes = [
    "Customer Service",
    "Audio",
    "Visual",
    "Bluetooth",
    "Microphone",
    "Battery Life",
    "Internet",
    "Performance/Speed",
    "Value/Price",
    "Ease of Use",
    "Comfortability/Fit",
    "Undetermined"
]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for id, class_ in enumerate(classes)}
model_path = "microsoft/deberta-v3-large"

tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/dyang/miniconda3/envs/NLP_env3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
# load in the database
with open("reviews_labeled.csv", "r") as f:
    reader = csv.reader(f)
    data = list(reader)[1:]

In [6]:
bad_data = [0]
def preprocess_function(example):
    if 'Review' not in example or 'Label' not in example:
        bad_data[0] += 1
        print(f"Bad data: {bad_data[0]}")
        return None

    text = example['Review']
    label = example['Label']

    labels = [0.0 for _ in classes]
    if label in class2id:
        labels[class2id[label]] = 1.0

    tokenized_sentence = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    return {
        "input_ids": tokenized_sentence['input_ids'],  # Adjusted to how your tokenizer returns values
        "attention_mask": tokenized_sentence['attention_mask'],
        "labels": labels
    }

dataset_arr = []
for row in data:
    if len(row) > 1 and row[1] in classes:
        processed = preprocess_function({"Review": row[0], "Label": row[1]})
        if processed:
            dataset_arr.append(processed)

print(len(dataset_arr))

18687


In [7]:
from sklearn.model_selection import train_test_split

# split data into training, validation, and testing
train_data, test_data = train_test_split(dataset_arr, test_size=0.05, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)

print(f"Train: {len(train_data)}")
print(f"Validation: {len(val_data)}")
print(f"Test: {len(test_data)}")

Train: 17752
Validation: 468
Test: 467


In [8]:
train_data_dict = {"input_ids": [], "attention_mask": [], "labels": []}
val_data_dict = {"input_ids": [], "attention_mask": [], "labels": []}
test_data_dict = {"input_ids": [], "attention_mask": [], "labels": []}

for data in train_data:
    train_data_dict["input_ids"].append(data["input_ids"])
    train_data_dict["attention_mask"].append(data["attention_mask"])
    train_data_dict["labels"].append(data["labels"])

for data in val_data:
    val_data_dict["input_ids"].append(data["input_ids"])
    val_data_dict["attention_mask"].append(data["attention_mask"])
    val_data_dict["labels"].append(data["labels"])

for data in test_data:
    test_data_dict["input_ids"].append(data["input_ids"])
    test_data_dict["attention_mask"].append(data["attention_mask"])
    test_data_dict["labels"].append(data["labels"])

train_dataset = Dataset.from_dict(train_data_dict)
val_dataset = Dataset.from_dict(val_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

tokenized_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [9]:
# Define a simple function to convert lists to tensors
def collate_fn(batch):
    return {key: torch.tensor([d[key] for d in batch]) for key in batch[0]}

# Creating DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)


In [15]:
# load the multi class classification model
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(classes), id2label=id2class, label2id=class2id, problem_type="multi_label_classification")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
def train(model, train_dataloader, val_dataloader, optimizer, training_args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Training on {device}")

    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

    best_val_loss = float('inf')

    for epoch in range(training_args.num_train_epochs):
        model.train()
        total_train_loss = 0
        train_progress_bar = tqdm(train_dataloader, total=len(train_dataloader),
                                  desc=f"Epoch {epoch + 1}/{training_args.num_train_epochs} Training")

        for i, batch in enumerate(train_progress_bar):
            optimizer.zero_grad()

            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            if (i + 1) % 40 == 0:
                print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item():.4f}")

        scheduler.step()

        if training_args.evaluation_strategy == 'epoch':
            model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for batch in tqdm(val_dataloader, desc="Validating"):
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                    labels = batch['labels'].to(device)
                    outputs = model(**inputs)
                    val_loss = criterion(outputs.logits, labels)
                    total_val_loss += val_loss.item()

            avg_val_loss = total_val_loss / len(val_dataloader)
            print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}")

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                if training_args.save_strategy == "best":
                    torch.save(model.state_dict(), os.path.join(training_args.output_dir, 'best_model.pth'))

        if training_args.save_strategy == "epoch":
            torch.save(model.state_dict(), os.path.join(training_args.output_dir, f'model_epoch_{epoch + 1}.pth'))

    if training_args.load_best_model_at_end and training_args.save_strategy == "best":
        model.load_state_dict(torch.load(os.path.join(training_args.output_dir, 'best_model.pth')))

    return model


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

training_args = TrainingArguments(

   output_dir="classification_beta",
   learning_rate=2e-4,
   per_device_train_batch_size=2,
   per_device_eval_batch_size=2,
   num_train_epochs=5,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = train(model, train_dataloader, val_dataloader, optimizer, training_args)

trainer.train()


/home/dyang/miniconda3/envs/NLP_env3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Training on cuda


Epoch 1/5 Training:   2%|▏         | 40/2219 [00:17<13:28,  2.70it/s] 

Epoch 1, Batch 40, Loss: 2.1599


Epoch 1/5 Training:   4%|▎         | 80/2219 [00:33<13:05,  2.72it/s]

...

Epoch 5/5 Training:  99%|█████████▉| 2200/2219 [14:15<00:06,  2.98it/s]

Epoch 5, Batch 2200, Loss: 0.0011


Validating: 100%|██████████| 59/59 [00:08<00:00,  7.08it/s]


Epoch 5, Validation Loss: 0.4653


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [10]:
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(classes), id2label=id2class, label2id=class2id, problem_type="multi_label_classification")
model.load_state_dict(torch.load('classification_beta/model_epoch_5.pth'))
model = model.to(device)
model.eval()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [11]:
criterion = torch.nn.CrossEntropyLoss()

total_test_loss = 0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        test_loss = criterion(outputs.logits, labels)
        total_test_loss += test_loss.item()

        # Compute accuracy
        predicted_labels = torch.max(outputs.logits, 1)[1]  # Gets the indices of max logit
        predicted_labels_one_hot = F.one_hot(predicted_labels, num_classes=len(classes)).to(device)

        # Ensure labels are in the correct dtype for comparison, they should be in the same dtype as 'predicted_labels_one_hot'
        labels = labels.to(torch.int64)  # Adjust dtype if necessary

        # Now calculate the accuracy
        correct_predictions += (predicted_labels_one_hot == labels).all(dim=1).sum().item()
        total_predictions += labels.size(0)

avg_test_loss = total_test_loss / len(test_dataloader)
accuracy = correct_predictions / total_predictions * 100

print(f"Average test loss: {avg_test_loss:.2f}")
print(f"Accuracy: {accuracy:.2f}%")


Testing: 100%|██████████| 59/59 [00:05<00:00, 10.75it/s]

Average test loss: 0.46
Accuracy: 88.65%


In [12]:
# custom input into the model
custom_text = "I wish this device would not run out of power so quickly"
print("\033[94m" + custom_text + "\033[0m")

tokenized_text = tokenizer(custom_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
tokenized_text = {k: v.to(device) for k, v in tokenized_text.items()}
output = model(**tokenized_text)

predicted_labels = torch.max(output.logits, 1)[1]
predicted_class = id2class[predicted_labels.item()]

print(f"Predicted class: {predicted_class}")

custom_text = "I can't hear a thing on this crappy headset"
print("\033[94m" + custom_text + "\033[0m")

tokenized_text = tokenizer(custom_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
tokenized_text = {k: v.to(device) for k, v in tokenized_text.items()}
output = model(**tokenized_text)

predicted_labels = torch.max(output.logits, 1)[1]
predicted_class = id2class[predicted_labels.item()]

print(f"Predicted class: {predicted_class}")

I wish this device would not run out of power so quickly
Predicted class: Battery Life
I can't hear a thing on this crappy headset
Predicted class: Audio
